In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import upscale_functions as uf
import re

In [2]:
mnfi_file = r'/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/mnfi_layers.nc'
meteo_file = r'/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/meteo_layers.nc'

mnfi = xr.open_dataset(mnfi_file)
meteo = xr.open_dataset(meteo_file)

# Create an empty dataset with the same coordinates and dimensions
fluxes = xr.Dataset(coords=mnfi.coords)
shape = (mnfi.dims['y'], mnfi.dims['x'])

In [3]:
# sorting and selecting only ika
datvar = []
for i in range(1,13):
    datvar.append(f'ika_{i}')
mnfi = mnfi[datvar]

In [4]:
# Loop through all 'ika_X_Y_Z' variables
for var_name in mnfi.data_vars:
    match = re.search(r'ika_(\d{1,2})$', var_name)  # Matches 1–2 digit numbers at the end
    if match:
        X = int(match.group(1))
        if X in ['1', '3', '5', '7', '9', '11']:
            Fert = 1
        else:
            Fert = 2
    
        if X in [1, 2, 5, 6, 9, 10]:
            Habitat = 0
        else:
            Habitat = 1
            
        Plant = 0  # Adjust if need

        
        # Compute fluxes
        GPP = uf.gpp_model(
            Age=mnfi[var_name],
            model=4,
            T=meteo['air_temperature'],
            Vpd=meteo['vapor_pressure_deficit'],
            Fert=Fert,
            Habitat=Habitat
        )

        RECO = uf.reco_model(
            Age=mnfi[var_name],
            model=3,
            T=meteo['air_temperature'],
            Plant=Plant,
            Fert=Fert
        )

        NEP_model = uf.nep_model(
            Age=mnfi[var_name],
            model=3,
            T=meteo['air_temperature'],
            Plant=Plant,
            Fert=Fert
        )

        NEP_balance = GPP - RECO

        # Store in the fluxes dataset
        suffix = X
        fluxes[f'GPP_{suffix}'] = GPP
        fluxes[f'RECO_{suffix}'] = RECO
        fluxes[f'NEP_{suffix}'] = NEP_model
        fluxes[f'BAL_{suffix}'] = NEP_balance

In [10]:
# Set keyword to filter variables
plot_only = 'GPP'  # or 'RECO', 'NEP', 'BAL', etc.
fig_file = f'/scratch/project_2000908/nousu/mNFI_to_NEP/FIGS/{plot_only}_plots.png'

# Filter variables from fluxes that contain the keyword
selected_vars = [v for v in fluxes.data_vars if plot_only in v.upper()]

# Determine layout
n_vars = len(selected_vars)
n_cols = 4
n_rows = int(np.ceil(n_vars / n_cols))

# Create figure and axes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 5 * n_rows))
axes = axes.flatten()

# Plot each selected variable
for i, var in enumerate(selected_vars):
    fluxes[var].plot(ax=axes[i], cmap="viridis", cbar_kwargs={'shrink': 0.7})
    axes[i].set_title(var)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

# Hide unused subplots
for j in range(n_vars, len(axes)):
    axes[j].axis("off")

# Save to file
plt.savefig(fig_file, dpi=300, bbox_inches='tight')

In [11]:
output_file = r'/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/carbon_fluxes.nc'
fluxes.to_netcdf(output_file)